In [1]:
import warnings
from fastai.vision import *
from fastai.callbacks import *
from fastai.vision.models import cadene_models 
warnings.filterwarnings('ignore')
path = Path('../data/histopathologic/')
sys.path.append("dev/"); from metric_utils import AUC

In [2]:
# create ensemble dir
os.makedirs(path/'ensemble', exist_ok=True)

In [3]:
preds_path = path/'preds'
subs_path = path/'submissions'
ens_path = path/'ensemble'

In [4]:
preds_best_of_dirs = (preds_path).ls(); preds_best_of_dirs

[PosixPath('../data/histopathologic/preds/best_of_resnet101'),
 PosixPath('../data/histopathologic/preds/best_of_densenet169'),
 PosixPath('../data/histopathologic/preds/best_of_densenet201'),
 PosixPath('../data/histopathologic/preds/best_of_resnet152'),
 PosixPath('../data/histopathologic/preds/best_of_vgg16_bn')]

In [5]:
(preds_path/'best_of_densenet169').ls()

[PosixPath('../data/histopathologic/preds/best_of_densenet169/fold0_preds.pkl'),
 PosixPath('../data/histopathologic/preds/best_of_densenet169/fold2_TTA_preds.pkl'),
 PosixPath('../data/histopathologic/preds/best_of_densenet169/fold1_preds.pkl'),
 PosixPath('../data/histopathologic/preds/best_of_densenet169/fold3_TTA_preds.pkl'),
 PosixPath('../data/histopathologic/preds/best_of_densenet169/fold4_preds.pkl'),
 PosixPath('../data/histopathologic/preds/best_of_densenet169/fold3_preds.pkl'),
 PosixPath('../data/histopathologic/preds/best_of_densenet169/fold2_preds.pkl'),
 PosixPath('../data/histopathologic/preds/best_of_densenet169/fold1_TTA_preds.pkl'),
 PosixPath('../data/histopathologic/preds/best_of_densenet169/fold0_TTA_preds.pkl'),
 PosixPath('../data/histopathologic/preds/best_of_densenet169/fold4_TTA_preds.pkl')]

### simple average

In [27]:
models = ['densenet169', 'densenet201', 'resnet101']

In [28]:
mean_label = np.mean([pd.read_csv(subs_path/f'best_of_{m}/{m}_avg_TTA.csv')['label'] for m
           in models], axis=0)

In [29]:
new_sub = pd.read_csv(subs_path/f'best_of_{models[0]}/{models[0]}_avg_TTA.csv')
new_sub['label'] = mean_label

In [30]:
fname = "_".join(models)
new_sub.to_csv(ens_path/f"avg_TTA_mean_{fname}", index=False)

In [31]:
sub_name = ens_path/f"avg_TTA_mean_{fname}"

In [32]:
sub_name, new_sub

(PosixPath('../data/histopathologic/ensemble/avg_TTA_mean_densenet169_densenet201_resnet101'),
                                              id     label
 0      0b2ea2a822ad23fdb1b5dd26653da899fbd2c0d5  0.036974
 1      95596b92e5066c5c52466c90b69ff089b39f2737  0.238685
 2      248e6738860e2ebcf6258cdc1f32f299e0c76914  0.001165
 3      2c35657e312966e9294eac6841726ff3a748febf  0.002457
 4      145782eb7caa1c516acbe2eda34d9a3f31c41fd6  0.264971
 5      725dabe6ecccc68b958a2c7dd75bcbf362c7cb03  0.802345
 6      aa0307865281d4484ddf8c637c348292968b93a7  0.545319
 7      f4e5dc9c949920f1b3362982e15e99bf6f3ef83b  0.371964
 8      95e08c9cedc28a9b4a86f4fc1e06c1972134be08  0.009611
 9      ee1f5345a8d4e28403b7e61b97b5f76c201ce9cb  0.050511
 10     14b1e9a72e27a35f13056db611feb3c8825916f4  0.869793
 11     4f7710b24b9b776f6bfc89fad5469592d18e5431  0.050666
 12     2b0668109c7c446c94d29e1b554016b486e84bde  0.012559
 13     8dbd712b5b5e86c18c97da3fb7845cfe23cdf1c5  0.001311
 14     0084317dbb33

In [33]:
cmd = "kaggle competitions submit -c histopathologic-cancer-detection -f {} -m {}_avg_TTA".format(sub_name, fname)
os.system(cmd), print(cmd)

kaggle competitions submit -c histopathologic-cancer-detection -f ../data/histopathologic/ensemble/avg_TTA_mean_densenet169_densenet201_resnet101 -m densenet169_densenet201_resnet101_avg_TTA


(0, None)

### simple rank average

### simple all average

In [17]:
csvs= []
for m  in models:
    csvs += (preds_path/f'best_of_{m}').ls()

In [30]:
csvs = [o for o in csvs if 'TTA' in o.stem];csvs

[PosixPath('../data/histopathologic/preds/best_of_densenet169/fold2_TTA_preds.pkl'),
 PosixPath('../data/histopathologic/preds/best_of_densenet169/fold3_TTA_preds.pkl'),
 PosixPath('../data/histopathologic/preds/best_of_densenet169/fold1_TTA_preds.pkl'),
 PosixPath('../data/histopathologic/preds/best_of_densenet169/fold0_TTA_preds.pkl'),
 PosixPath('../data/histopathologic/preds/best_of_densenet169/fold4_TTA_preds.pkl'),
 PosixPath('../data/histopathologic/preds/best_of_vgg16_bn/fold2_TTA_preds.pkl'),
 PosixPath('../data/histopathologic/preds/best_of_vgg16_bn/fold3_TTA_preds.pkl'),
 PosixPath('../data/histopathologic/preds/best_of_vgg16_bn/fold1_TTA_preds.pkl'),
 PosixPath('../data/histopathologic/preds/best_of_vgg16_bn/fold0_TTA_preds.pkl'),
 PosixPath('../data/histopathologic/preds/best_of_vgg16_bn/fold4_TTA_preds.pkl'),
 PosixPath('../data/histopathologic/preds/best_of_densenet201/fold2_TTA_preds.pkl'),
 PosixPath('../data/histopathologic/preds/best_of_densenet201/fold3_TTA_preds.pk

In [31]:
pd.read_pickle('../data/histopathologic/preds/best_of_densenet169/fold0_preds.pkl')

tensor([[8.5557e-04, 9.9914e-01],
        [8.8339e-01, 1.1661e-01],
        [9.9370e-01, 6.3002e-03],
        ...,
        [9.9839e-01, 1.6088e-03],
        [9.9041e-01, 9.5919e-03],
        [3.7081e-03, 9.9629e-01]])

In [32]:
mean_label = np.mean([to_np(pd.read_pickle(csv)[:,1]) for csv in csvs], axis=0)

In [33]:
new_sub = pd.read_csv(path/'sample_submission.csv')
new_sub['label'] = mean_label

In [34]:
new_sub

,id,label
0,0b2ea2a822ad23fdb1b5dd26653da899fbd2c0d5,0.995882
1,95596b92e5066c5c52466c90b69ff089b39f2737,0.477222
2,248e6738860e2ebcf6258cdc1f32f299e0c76914,0.050597
3,2c35657e312966e9294eac6841726ff3a748febf,0.109161
4,145782eb7caa1c516acbe2eda34d9a3f31c41fd6,0.079293
5,725dabe6ecccc68b958a2c7dd75bcbf362c7cb03,0.962757
6,aa0307865281d4484ddf8c637c348292968b93a7,0.002694
7,f4e5dc9c949920f1b3362982e15e99bf6f3ef83b,0.735044
8,95e08c9cedc28a9b4a86f4fc1e06c1972134be08,0.999680
9,ee1f5345a8d4e28403b7e61b97b5f76c201ce9cb,0.960282


In [35]:
fname = "all_simple_TTA_mean.csv"
new_sub.to_csv(ens_path/f"{fname}", index=False)

In [36]:
sub_name = ens_path/f"{fname}"; sub_name

PosixPath('../data/histopathologic/ensemble/all_simple_TTA_mean')

In [37]:
cmd = "kaggle competitions submit -c histopathologic-cancer-detection -f {} -m {}".format(sub_name, fname)
os.system(cmd)

0

In [38]:
cmd

'kaggle competitions submit -c histopathologic-cancer-detection -f ../data/histopathologic/ensemble/all_simple_TTA_mean -m all_simple_TTA_mean'